In [1]:
from mcstasscript.interface import instr, functions, plotter
import matplotlib
%matplotlib widget

## Demo of cryostat builder
This notebook contains a quick demo of the Cryostat class that makes the task of adding a cryostat model much less daunting. First an instrument object is created with just a source.

In [2]:
instrument = instr.McStas_instr("cryostat_test")

The following components are found in the work_directory / input_path:
     Union_sphere.comp
     Union_box.comp
     Single_crystal_process.comp
     Union_logger_2D_kf.comp
     Template_process.comp
     Union_conditional_standard.comp
     Union_logger_2D_space.comp
     Union_conditional_PSD.comp
     Union_master.comp
     Union_logger_2D_kf_time.comp
     Union_cylinder.comp
     Powder_process.comp
     Union_make_material.comp
     Incoherent_process.comp
     Union_logger_1D.comp
     Union_logger_3D_space.comp
     Union_logger_2DQ.comp
     Union_logger_2D_space_time.comp
These definitions will be used instead of the installed versions.


In [3]:
source = instrument.add_component("source", "Source_simple")
source.xwidth = 0.01
source.yheight = 0.01
source.focus_xw = 0.01
source.focus_yh = 0.01
source.dist = 2
source.E0 = 5
source.dE = 0.1

## Creating a cryostat
One use the Cryostat class to make a cryostat object. This object can be placed in the instrument file much like a component with *set_AT* and *set_ROTATED*. Here we place it 2 m after the source.

Then the different layers of the cryostat is added, starting with the smallest closest to the sample. A negative value for top or bottom thickness removes the cap in that end. After adding a layer, it can be accessed with the attribute *last_layer*, which allows adding a window. Here 4 layers are set up, 3 of which have windows.

In [4]:
orange_cryostat = functions.Cryostat("orange", instrument)
orange_cryostat.set_AT([0,0,2], source)

orange_cryostat.add_layer(inner_radius=70E-3/2, outer_radius=75E-3/2,
                          sample_to_bottom=83E-3, bottom_thickness=5E-3,
                          sample_to_top=200E-3, top_thickness=-1E-3,
                          material="Al", p_interact=0.2)
orange_cryostat.last_layer.add_window(outer_radius=73E-3/2, sample_to_top=44.42E-3, sample_to_bottom=88.2E-3)

orange_cryostat.add_layer(inner_radius=80E-3/2, outer_radius=81E-3/2,
                          sample_to_bottom=90E-3, bottom_thickness=2E-3,
                          sample_to_top=240E-3, top_thickness=-1E-3, p_interact=0.2)

orange_cryostat.add_layer(inner_radius=95E-3/2, outer_radius=99.5E-3/2,
                          sample_to_bottom=93E-3, bottom_thickness=6E-3,
                          sample_to_top=225E-3, top_thickness=9E-3, p_interact=0.2)
orange_cryostat.last_layer.add_window(outer_radius=97E-3/2, sample_to_top=52E-3, sample_to_bottom=100E-3)

orange_cryostat.add_layer(inner_radius=120E-3/2, outer_radius=127E-3/2,
                          sample_to_bottom=109E-3, bottom_thickness=11E-3,
                          sample_to_top=205E-3, top_thickness=22E-3, p_interact=0.2)
orange_cryostat.last_layer.add_window(outer_radius=125E-3/2, inner_radius=122E-3/2,
                                      sample_to_top=55.7E-3, sample_to_bottom=93.54E-3)

After the cryostat description is done one can optionally add Union loggers that show scattering intensity in space using the *add_spatial_loggers* method. Shows the cryostat from the 3 directions along axis, and a zoom in on a cut in zy that clearly shows all windows added. In addition its possible to record the scattering as a function of time, this is done with the *add_time_histogram* and *add_animation* methods, the first of which adds a simple histogram and the latter of which records spatial scattering in a number of time frames.

At the end it is necessary to run the *build* method, this assigns the appropriate priorites to each Union component used, and adds a Union_master component to the end. A sample could be added before the *build* method is called using Union components.

In [5]:
orange_cryostat.add_spatial_loggers()
orange_cryostat.add_time_histogram(t_min=0.00195, t_max=0.0024)
orange_cryostat.add_animation(t_min=0.00195, t_max=0.0024, n_frames=5)
orange_cryostat.build()

For further information on these methods, the built in help can be used shown below

In [6]:
help(orange_cryostat.add_animation)

Help on method add_animation in module mcstasscript.tools.cryostat:

add_animation(t_min=0, t_max=0.1, n_frames=10, d1='z', n1=300, d2='y', n2=300) method of mcstasscript.tools.cryostat.Cryostat instance
    Adds 2D_space_time logger that records the information necessary for animation
    
    Adds a 2D_space_time logger that records the requested number of frames, n_frames.
    The time span of both is from t_min to t_max. The default orientation is in the zy
    plane, but this can be chosen along with the desired resolution.
    Errors can happen with too many empty frames, so it is recommended to set
    t_min close to the first scattering time.
    Select the appropriate animation data from simulation output and use
    plotter.make_animation with a filename to save the animation as a gif.
    
    Parameters
    ----------
    
    t_min : float
        Lowest time recorded in [s]
    
    t_max : float
        Highest time recorded in [s]
    
    n_frames : int
        Number 

### Running the simulation
This is sufficient to run the simulation and see the resulting plots.

In [7]:
data = instrument.run_full_instrument(foldername="test_cryostat", increment_folder_name=True, ncount=1E7)

INFO: Using directory: "/Users/madsbertelsen/PaNOSC/McStasScript/github/McStasScript/examples/test_cryostat_7"
INFO: Regenerating c-file: cryostat_test.c
CFLAGS= -I@MCCODE_LIB@/share/
INFO: Recompiling: ./cryostat_test.out
In file included from Incoherent_process.comp:65:0:
./Union_functions.c: In function ‘write_tagging_tree’:
./Union_functions.c:1604:82: warning: passing argument 4 of ‘qsort’ from incompatible pointer type [enabled by default]
   qsort(total_history.saved_histories,total_history.used_elements,sizeof (struct saved_history_struct), Sample_compare_history_intensities);
                                                                                  ^
In file included from mccode-r.h:41:0:
/usr/include/stdlib.h:165:7: note: expected ‘int (*)(const void *, const void *)’ but argument is of type ‘int (*)(const struct saved_history_struct *, const struct saved_history_struct *)’
 void  qsort(void *__base, size_t __nel, size_t __width,
       ^
In file included from Incoher

### Plotting with interface
Recommend using log plot and orders of magnitude = 5 to see details. It may be necessary to refresh this cell after the simulation has been performed.

In [9]:
%matplotlib widget
plotter.interface(data)

/anaconda3/lib/python3.7/site-packages/matplotlib/axes/_base.py:3099: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=1.0, right=1.0
  self.set_xlim(upper, lower, auto=None)


## Future expansions
Some additional features are expected to be added to this system at a later point.

### Entry windows
Create a layer method to make entry windows, square and circular.

### Mounting plate
A cryostat usually has a mounting plate with a larger radius than the widest layer, could easily add such a feature.

### External sample
Create a cryostat method that takes an external sample component that is not in Union but incorperates this into the cryostat using the best practice method.